In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
import pandas as pd
import numpy as np
import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper

In [ ]:
np.random.seed(4242)

In [ ]:
# adv_df = pd.read_csv('adv_df.csv')

In [ ]:
adv_df = pd.read_csv('https://raw.githubusercontent.com/cmhamakawa/Data_Science_NBA_Project/master/Datasets/v2preprocessed_nba_dataset(1990-2020).csv')

In [ ]:
# christine_subset = adv_df[['Pos','SalStartYr', 'GS', 'MP', '2PA', '2P', 'eFG%', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'PF', 'PTS', 'years_of_exp','Salary']]

In [ ]:
# christine_subset = adv_df[['FG', 'FTA', 'AST', 'TRB', 'PTS', 'BLK', 'years_of_exp','Salary','Pos','SalStartYr','Player']]

In [ ]:
christine_subset = adv_df[['FG', 'FTA', 'AST', 'TRB', 'PTS', 'BLK', 'years_of_exp','Salary','Pos','SalStartYr','Player']]

In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
# l1 = list(christine_subset['Player'].unique())

In [ ]:
# l2 = [i.replace('*','') for i in l1]

In [ ]:
# np.array(l2)

In [ ]:
v1 = christine_subset.groupby(['Player','SalStartYr','FG', 'FTA', 'AST', 'TRB', 'PTS', 'BLK', 'years_of_exp','Pos']).agg({'Salary':sum}).reset_index()

In [ ]:
v1.head()

In [ ]:
v1['Player'] = v1['Player'].str.replace('*','')

In [ ]:
v1['index'] = v1.index

In [ ]:
def get_year_before(val):
    experience = v1.loc[v1['index'] == val, 'years_of_exp'].values[0]
    if experience == 1:
        return v1.loc[v1['index'] == val, 'Salary'].values[0]
    else:
        try:
            return v1.loc[v1['index'] == (val - 1), 'Salary'].values[0]
        except IndexError:
            return 999

In [ ]:
v1['salary_before'] = v1['index'].apply(get_year_before)

In [ ]:
v1.drop(columns = 'index', inplace = True)

In [ ]:
v1 = v1[v1['salary_before'] != 999]

In [ ]:
v1.head()

In [ ]:
v1.index = v1['Player']

In [ ]:
v1 = v1[v1['years_of_exp'] != 1]

In [ ]:
v1.head()

In [ ]:
# #plot all variables (excluding the target variable) against the target variable (sales)
# for i in list(adv_df.drop(columns = ['Player','StartYr','EndYr','SalEndYr','Salary','SalaryInfl','salary_cap_adjusted','salary_cap','Per_of_Salary_Cap']).columns):
#     sns.catplot(x = i, y = 'Salary', data = adv_df)

In [ ]:
christine_subset = v1

In [ ]:
christine_subset[christine_subset['Player'] == 'Aaron Gordon']['Salary'][0]

In [ ]:
#first quartile
q1 = christine_subset['Salary'].quantile(.25)

In [ ]:
#first quartile
q3 = christine_subset['Salary'].quantile(.75)

In [ ]:
#interquartile range
iqr = q3 - q1
iqr

In [ ]:
#lower limit for outliers, lower than 0 so no need to account for
lower = q1 - 1.5 * iqr
lower

In [ ]:
#upper limit for outliers
upper = q3 + 1.5 * iqr
upper

In [ ]:
# christine_subset = christine_subset[(christine_subset['Salary'] > christine_subset['Salary'].quantile(low)) & (christine_subset['Salary'] < christine_subset['Salary'].quantile(high))]

In [ ]:
christine_subset.shape

In [ ]:
# len(christine_subset[christine_subset['Cluster'] == 2])

In [ ]:
outliers = christine_subset[christine_subset['Salary'] >= upper]
outliers

In [ ]:
outliers.index

In [ ]:
# christine_subset = christine_subset[christine_subset['Salary'] < upper]

In [ ]:
# others = christine_subset.columns

In [ ]:
# features = [('SalStartYr', [sklearn.preprocessing.LabelBinarizer()]),('Pos', [sklearn.preprocessing.LabelBinarizer()])]
# for column, alias in zip(others, others):
#     stuff = ([column], sklearn.preprocessing.MinMaxScaler(), {'alias': alias})
#     features.append(stuff)

# mapper = DataFrameMapper(features,df_out = True)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
christine_subset['SalStartYr'] = le.fit_transform(christine_subset['SalStartYr'])
christine_subset['Player'] = le.fit_transform(christine_subset['Player'])
christine_subset['Pos'] = le.fit_transform(christine_subset['Pos'])


In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as pl

In [ ]:
# to_elbow = fixed_df

In [ ]:
to_elbow = christine_subset.drop(columns = ['salary_before'], axis = 1)

In [ ]:
# to_elbow = christine_subset[['salary_before']]

In [ ]:
distortions = []
inertias = []

K = range(1, 10)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(to_elbow)
    kmeanModel.fit(to_elbow)
 
    distortions.append(sum(np.min(cdist(to_elbow, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / to_elbow.shape[0])
    
    inertias.append(kmeanModel.inertia_)

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=3,random_state=1)

In [ ]:
kmeans.fit(to_elbow)

In [ ]:
to_elbow['Cluster'] = kmeans.labels_

In [ ]:
to_elbow['Cluster'].unique()

In [ ]:
to_elbow.shape

In [ ]:
christine_subset['Cluster'] = to_elbow['Cluster']

In [ ]:
# christine_subset['SalStartYr'] = le.fit_transform(christine_subset['SalStartYr'])
# christine_subset['Tm'] = le.fit_transform(christine_subset['Tm'])

In [ ]:
# christine_subset = christine_subset.join(fixed_df.iloc[:,0:20])

In [ ]:
christine_subset.head()

# How Well Can I Predict Clusters?

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# christine_subset.shape

In [ ]:
X = christine_subset.drop(columns = ['Cluster','Salary'], axis = 1)
y = christine_subset[['Cluster']]

In [ ]:
# X = christine_subset.drop(columns = ['Cluster','Salary','salary_before','Player','Pos','SalStartYr'], axis = 1)
# y = christine_subset[['Cluster']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)

# Logistic Regression

In [ ]:
LR = LogisticRegression()
cross_val_score(LR, X_train, y_train, cv = 5).mean()

In [ ]:
c_vals = np.linspace(0, 3, 10)
c_vals

In [ ]:
fig, ax = plt.subplots(1)

best_score = 0
best_c = 0

# c_vals from 0-10
c_vals = np.linspace(0, 3, 20)

# loop through possible c_vals
for c_val in c_vals:
    
    # evaluate the model with the current c_val
    LR = LogisticRegression(C = c_val)
    cv_score = cross_val_score(LR, X_train, y_train, cv=5).mean()
    ax.scatter(c_val, cv_score, color = "black")
    
    # check if we've beaten our previous score
    if cv_score > best_score:
        best_c = c_val
        best_score = cv_score

l = ax.set(title = "Best C : " + str(best_c),
       xlabel = "C", 
       ylabel = "CV Score")
ax.scatter(best_c, best_score, color = "red")

In [ ]:
LR = LogisticRegression(C = best_c)
LR.fit(X_train, y_train)
LR.score(X_test, y_test)

# Decision Trees

In [ ]:
T = DecisionTreeClassifier(max_depth = 2)
cross_val_score(T, X_train, y_train, cv=5).mean()

In [ ]:
fig, ax = plt.subplots(1)

best_score = 0

# cross validation for each of the possible model depths 1-30
for d in range(1,30):
    T = DecisionTreeClassifier(max_depth = d)
    # Average cv_score acheived in cross validation
    cv_score = cross_val_score(T, X_train, y_train, cv=10).mean()
    # plot avg cv_score (in black)
    ax.scatter(d, cv_score, color = "black")
    # identify the best possible depth by cv_score
    if cv_score > best_score:
        best_depth = d
        best_score = cv_score

#highlight the best depth, best score value in red
ax.scatter(best_depth, best_score, color = "red")

# show best depth in title 
# plot tells us the depth that acheived the best score under cross validation 
# we now have a good estimate about the optimal depth/complexity of model
l = ax.set(title = "Best Depth : " + str(best_depth),
       xlabel = "Depth", 
       ylabel = "CV Score")

In [ ]:
T = DecisionTreeClassifier(max_depth = best_depth)
T.fit(X_train, y_train)
T.score(X_train, y_train), T.score(X_test, y_test)

In [ ]:
# import pickle
# decision_tree_model_pkl = open('veterans_dt.pkl', 'wb')
# pickle.dump(T, decision_tree_model_pkl)
# decision_tree_model_pkl.close()

In [ ]:
# import MLPClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
N = MLPClassifier(verbose = True, max_iter = 1000).fit(X_train, y_train)

In [ ]:
N.predict(X_test)

In [ ]:
N.score(X_test, y_test)

# Predicting Salary

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import seaborn as sns

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
# cluster_0 = christine_subset[christine_subset['Cluster'] == 0]
# salary_0 = cluster_0['Salary']

In [ ]:
# others = christine_subset.drop(columns = 'Salary', axis = 1).columns

In [ ]:
# features = [('Salary', None)]
# for column, alias in zip(others, others):
#     stuff = ([column], sklearn.preprocessing.MinMaxScaler(), {'alias': alias})
#     features.append(stuff)

# mapper = DataFrameMapper(features,df_out = True)

# christine_subset = mapper.fit_transform(christine_subset.copy())

In [ ]:
christine_subset.head()

In [ ]:
# christine_subset = christine_subset[christine_subset['Salary'] >= upper]

In [ ]:
# christine_subset = christine_subset[christine_subset['Cluster'] == 2]

In [ ]:
#create training and test sets, encode
X = christine_subset.drop(columns = ['Salary'], axis=1)
y = christine_subset['Salary']
X.head()

In [ ]:
#splitting training and testing sets, 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
#base parameters
#optimize both bias and variance
params = {
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective':'reg:squarederror',
}

params['eval_metric'] = 'rmse'
num_boost_round = 999

In [ ]:
#initial run
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
#tabular data
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=10
)
cv_results

In [ ]:
#tune max_depth and min_child_weight
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(1, 20)
    for min_child_weight in range(1, 20)
]

In [ ]:
#iterate through combinations of parameter values and get lowest value
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [ ]:
#set parameter values to most optimal values
params['max_depth'] = best_params[0]
params['min_child_weight'] =  best_params[1]

In [ ]:
#tune subsample and colsample
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1, 11)]
    for colsample in [i/10. for i in range(1, 11)]
]

In [ ]:
#iterate through combinations of parameter values and get lowest value
min_rmse = float("Inf")
best_params = None

for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [ ]:
#set parameter values to optimal values
params['subsample'] = best_params[0]
params['colsample_bytree'] = best_params[1]

In [ ]:
#iterate through combinations of parameter values and get lowest value
min_rmse = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    
    params['eta'] = eta
    
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics=['rmse'],
        early_stopping_rounds=10
    )
    
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
print("Best params: {}, RMSE: {}".format(best_params, min_rmse))

In [ ]:
#set parameter value to optimal value
params['eta'] = best_params

In [ ]:
#train model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best RMSE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
#train model to optimal iteration
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [ ]:
#RMSE score
mean_squared_error(best_model.predict(dtest), y_test, squared = False)

In [ ]:
# overall: 1322826.5681605006

In [ ]:
# outliers: 2519363.4977930584

In [ ]:
# Cluster 0: 1021782.2753956259

In [ ]:
# Cluster 1: 1617855.6924153203

In [ ]:
# Cluster 2: 3138775.1092300224

In [ ]:
#save model
best_model.save_model("overall_veteran.model")

In [ ]:
#load model and predict
loaded_model = xgb.Booster()
loaded_model.load_model("overall_veteran.model")
predictions = [round(i) for i in loaded_model.predict(xgb.DMatrix(X_test))]
predictions

In [ ]:
#RMSE score
mean_squared_error(predictions, y_test, squared = False)

In [ ]:
# 1322826.571219358

In [ ]:
data = pd.DataFrame(np.column_stack([predictions, y_test]), columns = ['predictions','y_test'])

In [ ]:
data['index'] = data.index

In [ ]:
data = data[0:100]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data['index'], y=data['predictions'],
                    mode='lines',
                    name='predicted'))

fig.add_trace(go.Scatter(x=data['index'], y=data['y_test'],
                    mode='lines',
                    name='observed'))

fig.update_layout(title = 'Veteran Numbers (100 Players)')
fig.update_xaxes(title = 'Player')
fig.update_yaxes(title = 'Salary ($)')

fig.write_json("veteran_RMSE.json") 


fig.show()

# Testing Individual Values

In [ ]:
christine_subset.to_csv('veterans_christine.csv', index = True, index_label = True)

In [ ]:
# ex = pd.read_csv('veterans_christine.csv')
# ex.head()

In [ ]:
# l = ex[(ex['True'] == 'A.C. Green') & (ex['years_of_exp'] == 7)]
# float(l['Player'])

In [ ]:
# # corresponds to the 2020 season
# test = christine_subset[(christine_subset['SalStartYr'] == 29)]

In [ ]:
# test.head()

In [ ]:
# x = pd.DataFrame([test.loc['Aaron Gordon']])
# x

In [ ]:
# x.drop(columns = ['Salary','Cluster'], axis = 1)

In [ ]:
# x['Cluster'] = T.predict(x.drop(columns = ['Salary','Cluster'], axis = 1))
# x

In [ ]:
# to_input = x.drop(columns = ['Salary'], axis = 1)
# to_input

In [ ]:
# christine_subset[(christine_subset.index == 'Stephen Curry') & (christine_subset['years_of_exp'] == 11)]

In [ ]:
# final_df = pd.DataFrame(to_input.values.tolist())

In [ ]:
# loaded_model = xgb.Booster()
# loaded_model.load_model('outliers_veteran.model')

In [ ]:
# predictions = np.array([i for i in loaded_model.predict(xgb.DMatrix(final_df))])
# predictions